# A complete profile for sentiment analysis model

![picture](https://media.springernature.com/original/springer-static/image/chp%3A10.1007%2F978-981-15-0029-9_7/MediaObjects/475950_1_En_7_Fig2_HTML.png)

## Models Ideas:
1. Use of custom models for different categories (tech, food, books,...) to be automatically (using context classfication) or manually selected(by the client). *(different datasets applied)*
2. Run multiple models per dataset and derive weighted average results.
3. Developing a layered classification **use *fast/slow* classification** (divide the dataset using confidence index to strong and weak groups; the weak group will be analysed further using Roberta model).
4. Aspect based analysis **(attach sentiment to specific aspects rather than sentence/opinion)** and word cloud **(for word frequencies)** to show insights of the reviews. (Amazon comprehend model)
5. Use of lemmatization, opinion unit extractor, subjectivity index and multiclass classification(love, sad, angry,...) for better accuracy and data enrichment.
6. Test of a sent-ngrams lexion sentiment analysis **(SO-CAL)**.
7. Use of client dataset to fine-tune the model. (Ideation phase)

## Transformers Compararison:

In [5]:
df


,Model Name,Base Model,Pooling,Training Data,STSb Performance (Higher = Better),Speed (Sent. / Sec on V100 GPU),Comments
0,stsb-mpnet-base-v2,mpnet-base,Mean Pooling,NLI+STSb,88.57,2800,NaN
1,stsb-roberta-base-v2,roberta-base,Mean Pooling,NLI+STSb,87.21,2300,NaN
2,stsb-distilroberta-base-v2,distilroberta-base,Mean Pooling,NLI+STSb,86.41,4000,NaN
3,nli-mpnet-base-v2,mpnet-base,Mean Pooling,NLI,86.53,2800,NaN
4,stsb-roberta-large,roberta-base,Mean Pooling,NLI+STSb,86.39,830,NaN
5,nli-roberta-base-v2,roberta-base,Mean Pooling,NLI,85.54,2300,NaN
6,stsb-roberta-base,roberta-base,Mean Pooling,NLI+STSb,85.44,2300,NaN
7,stsb-bert-large,bert-large-uncased,Mean Pooling,NLI+STSb,85.29,830,NaN
8,stsb-distilbert-base,distilbert-base-uncased,Mean Pooling,NLI+STSb,85.16,4000,NaN
9,stsb-bert-base,bert-base-uncased,Mean Pooling,NLI+STSb,85.14,2300,NaN


## Datasets used:
1. Twitter airline 
2. IMDB 
3. Yelp (preprocessing phase)
4. Amazon 
5. 140sentiment twitter

## Implementation:

### Imports:

In [ ]:
# Install the transformers library
!pip install transformers
!pip install vaderSentiment
!pip install flair

In [ ]:
from textblob import TextBlob as tb
from textblob.sentiments import NaiveBayesAnalyzer
from textblob import Blobber
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from flair.data import Sentence
from flair.models import TextClassifier
from sklearn import metrics
import pandas as pd
from transformers import pipeline
from sklearn.utils import shuffle
import time
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
# only for colab
from google.colab import drive
drive.mount('/content/drive')
import io
from google.colab import files
import numpy as np 
import pandas as pd
uploaded=files.upload()
df=pd.read_excel(io.BytesIO(uploaded['SentenceTransformer Pretrained Models.xlsx']))

Mounted at /content/drive


### libraries implementation:

In [ ]:
def metric(true,predict):
    analytics=[]
    #metrics.classification_report(true,predict)
    analytics.append(round(metrics.accuracy_score(true,predict),2))
    analytics.append(round(metrics.precision_score(true,predict,average='weighted'),2))
    analytics.append(round(metrics.recall_score(true,predict,average='weighted'),2))
    analytics.append(round(metrics.f1_score(true,predict,average='weighted'),2))
    return analytics

In [ ]:
# class modelDataset:
#     def __init__(self, tokenized_texts):
#         self.tokenized_texts = tokenized_texts

#     def __len__(self):
#         return len(self.tokenized_texts["input_ids"])

#     def __getitem__(self, idx):
#         return {k: v[idx] for k, v in self.tokenized_texts.items()}

In [ ]:
def textblobPattern(text):
    sentiment = []
    start=time.time()
    for sentence in text:
        sent = tb(sentence).polarity
        if sent > 0:
            sentiment.append(1)
        elif sent < 0:
            sentiment.append(-1)
        else:
            sentiment.append(0)
    end=time.time()
    tm=(end-start)/len(text)
    return tm,sentiment


In [ ]:
def textblobNB(text):
    sentimentV, sentimentT = [], []
    tbnb = Blobber(analyzer=NaiveBayesAnalyzer())
    for sentence in text:
        ts = tbnb(sentence).sentiment
    return sentimentV, sentimentT


In [ ]:
def vader(text):
    sentiment = []
    analyzer = SentimentIntensityAnalyzer()
    start=time.time()
    for sentence in text:
        vs = analyzer.polarity_scores(sentence)['compound']
        if (vs > 0.5):
            sentiment.append(1)
        elif (vs < -0.5):
            sentiment.append(-1)
        else:
            sentiment.append(0)
    end = time.time()
    tm = (end - start) / len(text)
    return tm, sentiment


In [ ]:
def flair(text):
    classifier = TextClassifier.load('sentiment-fast')
    # sentences = [Sentence(t) for t in text]
    sentiment = []
    start=time.time()
    for phrase in text:
        sentence = Sentence(phrase)
        classifier.predict(sentence, mini_batch_size=32)
        sentiment.append(1 if sentence.labels[0].value == 'POSITIVE' else -1)
    end = time.time()
    tm = (end - start) / len(text)
    return tm, sentiment


In [ ]:
def RoBerta_large(text):
    # tokenizer = AutoTokenizer.from_pretrained("siebert/sentiment-roberta-large-english")
    # model = AutoModelForSequenceClassification.from_pretrained("siebert/sentiment-roberta-large-english")
    # trainer = Trainer(model=model)
    # tokenized_texts = tokenizer(text,truncation=True,padding=True)
    # pred_dataset = modelDataset(tokenized_texts)
    # predictions = trainer.predict(pred_dataset)
    # preds = predictions.predictions.argmax(-1)
    # return [-1 if x==0 else 1 for x in preds]
    sentiment = []
    sentiment_analysis = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english")
    start=time.time()
    for x in text:
        sentiment.append(1 if sentiment_analysis(x, truncation=True, padding=True)[0]['label'] == 'POSITIVE' else -1)
    end = time.time()
    tm = (end - start) / len(text)
    return tm, sentiment


In [ ]:

def bert_base(text):
    sentiment = []
    sentiment_analysis = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
    start=time.time()
    for x in text:
        sent=  (sentiment_analysis(x, truncation=True, padding=True)[0]['label'])
        if sent== "1 star":
                sentiment.append(-1)
        elif sent== "2 stars":
                sentiment.append(-1)
        elif sent == "3 stars":
                sentiment.append(0)
        elif sent =="4 stars":
                sentiment.append(1)
        elif sent =="5 stars":
                sentiment.append(1)
    end = time.time()
    tm = (end - start) / len(text)
    return tm, sentiment


### Model testing: (for each dataset)

In [ ]:
def UsAirTw():
    file_name = "Datasets/usAir_tweets.csv"
    text_column = "text"
    df = pd.read_csv(file_name)
    df=shuffle (df)
    df=df[:10000]
    true = df["sentiment"]
    pred_texts = df[text_column].dropna().astype('str').tolist()
    print("Metrics for usairline_tweets Dataset:")
    time, pred = textblobPattern(pred_texts)
    textblobM = metric(true, pred)
    print(f"The textblob metrics:\n accuracy={textblobM[0]},precision={textblobM[1]},recall={textblobM[2]},f1 score ={textblobM[3]},time per sentiment= {time}")
    time, pred = vader(pred_texts)
    vaderM = metric(true, pred)
    print(f"The Vader metrics:\n accuracy={vaderM[0]},precision={vaderM[1]},recall={vaderM[2]},f1 score ={vaderM[3]},time per sentiment= {time}")
    time, pred = flair(pred_texts)
    flairM = metric(true, pred)
    print(f"The flair metrics:\n accuracy={flairM[0]},precision={flairM[1]},recall={flairM[2]},f1 score ={flairM[3]},time per sentiment= {time}")
    time, pred = RoBerta_large(pred_texts)
    robertaM = metric(true, pred)
    print(f"The Roberta large metrics:\n accuracy={robertaM[0]},precision={robertaM[1]},recall={robertaM[2]},f1 score ={robertaM[3]},time per sentiment= {time}")
    time, pred = bert_base(pred_texts)
    Bert = metric(true, pred)
    print(f"The bert multilang metrics:\n accuracy={Bert[0]},precision={Bert[1]},recall={Bert[2]},f1 score ={Bert[3]},time per sentiment= {time}")

In [ ]:

def yelp():
    file_name = "Datasets/yelp.csv"
    text_column = "text"
    df = pd.read_csv(file_name)
    df=shuffle (df)
    df=df[:10000]
    true = df["sent"]
    pred_texts = df[text_column].dropna().astype('str').tolist()
    print("Metrics for yelp sentiment:")
    time, pred = textblobPattern(pred_texts)
    textblobM = metric(true, pred)
    print(f"The textblob metrics:\n accuracy={textblobM[0]},precision={textblobM[1]},recall={textblobM[2]},f1 score ={textblobM[3]},time per sentiment= {time}")
    time, pred = vader(pred_texts)
    vaderM = metric(true, pred)
    print(f"The Vader metrics:\n accuracy={vaderM[0]},precision={vaderM[1]},recall={vaderM[2]},f1 score ={vaderM[3]},time per sentiment= {time}")
    time, pred = flair(pred_texts)
    flairM = metric(true, pred)
    print(f"The flair metrics:\n accuracy={flairM[0]},precision={flairM[1]},recall={flairM[2]},f1 score ={flairM[3]},time per sentiment= {time}")
    time, pred = RoBerta_large(pred_texts)
    robertaM = metric(true, pred)
    print(f"The Roberta large metrics:\n accuracy={robertaM[0]},precision={robertaM[1]},recall={robertaM[2]},f1 score ={robertaM[3]},time per sentiment= {time}")
    time, pred = bert_base(pred_texts)
    Bert = metric(true, pred)
    print(f"The bert multilang metrics:\n accuracy={Bert[0]},precision={Bert[1]},recall={Bert[2]},f1 score ={Bert[3]},time per sentiment= {time}")

In [ ]:
def Imdb():
    file_name = "Datasets/Imdb.csv"
    text_column = "review"
    df = pd.read_csv(file_name)
    df=shuffle (df)
    df=df[:10000]
    true = df["sentiment"]
    pred_texts = df[text_column].dropna().astype('str').tolist()
    print("Metrics for IMDB dataset:")
    time, pred = textblobPattern(pred_texts)
    textblobM = metric(true, pred)
    print(f"The textblob metrics:\n accuracy={textblobM[0]},precision={textblobM[1]},recall={textblobM[2]},f1 score ={textblobM[3]},time per sentimen= t{time}")
    time, pred = vader(pred_texts)
    vaderM = metric(true, pred)
    print(f"The Vader metrics:\n accuracy={vaderM[0]},precision={vaderM[1]},recall={vaderM[2]},f1 score ={vaderM[3]},time per sentiment= {time}")
    time, pred = flair(pred_texts)
    flairM = metric(true, pred)
    print(f"The flair metrics:\n accuracy={flairM[0]},precision={flairM[1]},recall={flairM[2]},f1 score ={flairM[3]},time per sentiment= {time}")
    time, pred = RoBerta_large(pred_texts)
    robertaM = metric(true, pred)
    print(f"The Roberta large metrics:\n accuracy={robertaM[0]},precision={robertaM[1]},recall={robertaM[2]},f1 score ={robertaM[3]},time per sentiment= {time}")
    time, pred = bert_base(pred_texts)
    Bert = metric(true, pred)
    print(f"The bert multilang metrics:\n accuracy={Bert[0]},precision={Bert[1]},recall={Bert[2]},f1 score ={Bert[3]},time per sentiment= {time}")

In [ ]:
def t140sentiment():
    file_name = "/content/drive/MyDrive/140sentiment.csv"
    text_column = "text"
    df = pd.read_csv(file_name)
    df=shuffle (df)
    df=df[:10000]
    true = df["sent"]
    pred_texts = df[text_column].dropna().astype('str').tolist()
    print("Metrics for sentiment140 dataset:")
    time, pred = textblobPattern(pred_texts)
    textblobM = metric(true, pred)
    print(f"The textblob metrics:\n accuracy={textblobM[0]},precision={textblobM[1]},recall={textblobM[2]},f1 score ={textblobM[3]},time per sentiment= {time}")
    time, pred = vader(pred_texts)
    vaderM = metric(true, pred)
    print(f"The Vader metrics:\n accuracy={vaderM[0]},precision={vaderM[1]},recall={vaderM[2]},f1 score ={vaderM[3]},time per sentiment= {time}")
    time, pred = flair(pred_texts)
    flairM = metric(true, pred)
    print(f"The flair metrics:\n accuracy={flairM[0]},precision={flairM[1]},recall={flairM[2]},f1 score ={flairM[3]},time per sentiment= {time}")
    time, pred = RoBerta_large(pred_texts)
    robertaM = metric(true, pred)
    print(f"The Roberta large metrics:\n accuracy={robertaM[0]},precision={robertaM[1]},recall={robertaM[2]},f1 score ={robertaM[3]},time per sentiment= {time}")
    time, pred = bert_base(pred_texts)
    Bert = metric(true, pred)
    print( f"The bert multilang metrics:\n accuracy={Bert[0]},precision={Bert[1]},recall={Bert[2]},f1 score ={Bert[3]},time per sentiment= {time}")

In [ ]:
def amazon():
    file_name = "Datasets/amazonFoods.csv"
    text_column = "Text"
    df = pd.read_csv(file_name)
    df=shuffle (df)
    df=df[:10000]
    true = df["Score"]
    pred_texts = df[text_column].dropna().astype('str').tolist()
    print("Metrics for Amazon food dataset:")
    time,pred=textblobPattern(pred_texts)
    textblobM = metric(true, pred)
    print(f"The textblob metrics:\n accuracy={textblobM[0]},precision={textblobM[1]},recall={textblobM[2]},f1 score ={textblobM[3]},time per sentiment= {time}")
    time,pred=vader(pred_texts)
    vaderM = metric(true, pred)
    print(f"The Vader metrics:\n accuracy={vaderM[0]},precision={vaderM[1]},recall={vaderM[2]},f1 score ={vaderM[3]},time per sentiment= {time}")
    time,pred=flair(pred_texts)
    flairM = metric(true, pred)
    print(f"The flair metrics:\n accuracy={flairM[0]},precision={flairM[1]},recall={flairM[2]},f1 score ={flairM[3]},time per sentiment= {time}")
    time,pred=RoBerta_large(pred_texts)
    robertaM = metric(true, pred)
    print(f"The Roberta large metrics:\n accuracy={robertaM[0]},precision={robertaM[1]},recall={robertaM[2]},f1 score ={robertaM[3]},time per sentiment= {time}")
    time,pred=bert_base(pred_texts)
    Bert = metric(true, pred)
    print(f"The bert multilang metrics:\n accuracy={Bert[0]},precision={Bert[1]},recall={Bert[2]},f1 score ={Bert[3]},time per sentiment= {time}")




### Main:

In [ ]:

    UsAirTw()
    Imdb()
    t140sentiment()
    amazon()
    yelp()

## Results:
### for UsAirlineTweets:
- The textblob metrics:
 - accuracy=0.47, precision=0.68, recall=0.47, f1 score =0.48,time per sentiment = 0.0001
- The Vader metrics:
 - accuracy=0.41, precision=0.73, recall=0.41, f1 score =0.4,time per sentiment = 0.0001
- The flair metrics:
 - accuracy=0.67, precision=0.58, recall=0.67, f1 score =0.61 ,time per sentiment = 0.001
- The Roberta large metrics:
 - accuracy=0.75, precision=0.62, recall=0.75, f1 score =0.67,time per sentiment = 0.0173
- The bert multilang metrics:
 - accuracy=0.66, precision=0.64, recall=0.66, f1 score =0.64,time per sentiment = 0.071
 
### for imdb:
- The textblob metrics:
 - accuracy=0.69,precision=0.76,recall=0.69,f1 score =0.67,time per sentimenT= 0.0015
- The Vader metrics:
 - accuracy=0.64,precision=0.75,recall=0.64,f1 score =0.67,time per sentiment= 0.0064
- The flair metrics:
 - accuracy=0.9,precision=0.9,recall=0.9,f1 score =0.9,time per sentiment= 0.039
- The Roberta large metrics:
 - accuracy=0.96,precision=0.96,recall=0.96,f1 score =0.96,time per sentiment= 1.29
- The bert multilang metrics:
 - accuracy=0.78,precision=0.92,recall=0.78,f1 score =0.85,time per sentiment= 0.59

### for Amazon:
- The textblob metrics:
 - accuracy=0.79,precision=0.73,recall=0.79,f1 score =0.75,time per sentiment= 0.001
- The Vader metrics:
 - accuracy=0.75,precision=0.79,recall=0.75,f1 score =0.75,time per sentiment= 0.001
- The flair metrics:
 - accuracy=0.84,precision=0.82,recall=0.84,f1 score =0.82,time per sentiment= 0.013
- The Roberta large metrics:
 - accuracy=0.89,precision=0.84,recall=0.89,f1 score =0.86,time per sentiment= 0.97
- The bert multilang metrics:
 - accuracy=0.85,precision=0.88,recall=0.85,f1 score =0.86,time per sentiment= 0.29

### for 140sentiment:
- The textblob metrics:
 - accuracy=0.44,precision=0.7,recall=0.44,f1 score =0.52,time per sentiment= 0.00026
- The Vader metrics:
 - accuracy=0.3,precision=0.79,recall=0.3,f1 score =0.41,time per sentiment= 0.0001
- The flair metrics:
 - accuracy=0.7,precision=0.7,recall=0.7,f1 score =0.7,time per sentiment= 0.0031
-The Roberta large metrics:
 - accuracy=0.75,precision=0.76,recall=0.75,f1 score =0.75,time per sentiment= 0.401
- The bert multilang metrics:
 - accuracy=0.63,precision=0.72,recall=0.63,f1 score =0.67,time per sentiment= 0.112
 
### for yelp:
- The textblob metrics:
 - accuracy=0.68,precision=0.78,recall=0.68,f1 score =0.65,time per sentiment= 0.001
- The Vader metrics:
 - accuracy=0.62,precision=0.83,recall=0.62,f1 score =0.63,time per sentiment= 0.002
- The flair metrics:
 - accuracy=0.93,precision=0.93,recall=0.93,f1 score =0.93,time per sentiment= 0.022
- The Roberta large metrics:
 - accuracy=0.97,precision=0.97,recall=0.97,f1 score =0.97,time per sentiment= 0.92
- The bert multilang metrics:
 - accuracy=0.85,precision=0.94,recall=0.85,f1 score =0.89,time per sentiment= 0.42

## Topics covered:
- Textblob - vader - flair libraries
- Text operations: lemmatization - tokenization - vectorization - wordnet - tagging - n-gram 
- Machine learning concepts: vector space model, k-means clustering,[ Naive Bayes, k-NN, SVM] classifiers, decision tree - random forest - transformers (word2vec and wordtree of stanford).
- Technologies: Jupyter notebook - Google colab
- Dataset handeling: dataset preprocessing
- Sentiment analysis approaches
- Handeling multiple  Deeplearning models: roBERTa - BERT - [GloVe - Fasttext - torchtext]

## References:
- https://julsimon.medium.com/a-quick-look-at-natural-language-processing-with-amazon-comprehend-238b8d9ec11d (REVIEW VIP)
- https://neptune.ai/blog/sentiment-analysis-python-textblob-vs-vader-vs-flair
- https://towardsdatascience.com/customer-churn-accuracy-a-4-6-increase-with-feature-engineering-29bcb1b1ee8f (REVIEW)
- https://www.analyticsvidhya.com/blog/2021/01/sentiment-analysis-vader-or-textblob/
- https://pythonprogramming.net/sentiment-analysis-python-textblob-vader/
- https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664
- https://medium.com/geekculture/what-nlp-library-you-should-use-for-your-sentimental-analysis-project-bef6b357a6db
- https://towardsdatascience.com/sentiment-analysis-comparing-3-common-approaches-naive-bayes-lstm-and-vader-ab561f834f89
-https://www.digitalocean.com/community/tutorials/how-to-perform-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk
- https://medium.com/geekculture/what-nlp-library-you-should-use-for-your-sentimental-analysis-project-bef6b357a6db
- https://stackoverflow.com/questions/10059594/a-simple-explanation-of-naive-bayes-classification
****
* N-grams rule based model
- https://www.sciencedirect.com/science/article/pii/S095741741830143X
- https://github.com/sfu-discourse-lab/SO-CAL(to be reviewed)
- https://towardsdatascience.com/text-analysis-basics-in-python-443282942ec5
****
- https://julsimon.medium.com/a-quick-look-at-natural-language-processing-with-amazon-comprehend-238b8d9ec11d
-https://towardsdatascience.com/text-classification-with-state-of-the-art-nlp-library-flair-b541d7add21f
- https://machinelearningmastery.com/k-fold-cross-validation/
-https://medium.com/analytics-vidhya/top-5-unknown-sentiment-analysis-projects-on-github-to-help-you-through-your-nlp-projects-8d8f195e80fc
****
- https://towardsdatascience.com/sentiment-analysis-comparing-3-common-approaches-naive-bayes-lstm-and-vader-ab561f834f89
- https://towardsdatascience.com/aspect-based-sentiment-analysis-using-spacy-textblob-4c8de3e0d2b9
- https://towardsdatascience.com/text-classification-with-state-of-the-art-nlp-library-flair-b541d7add21f
- https://github.com/nahida-uap/Accuracy-improvement-of-Textblob-based-on-Supervised-learning-method
### transformers:
- https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4
- https://www.hindawi.com/journals/mpe/2021/5536560/
- https://hcis-journal.springeropen.com/articles/10.1186/s13673-017-0116-3
- https://paperswithcode.com/task/sentiment-analysis
- https://theappsolutions.com/blog/development/sentiment-analysis/
- https://www.linkedin.com/pulse/best-ai-algorithms-sentiment-analysis-muktabh-mayank/
- https://medium.com/inside-machine-learning/what-is-a-transformer-d07dd1fbec04
- https://neptune.ai/blog/document-classification-small-datasets
- https://www.marktechpost.com/2021/09/09/facebook-ai-introduces-gslm-generative-spoken-language-model-a-textless-nlp-model-that-breaks-free-completely-of-the-dependence-on-text-for-training/
- https://towardsdatascience.com/what-is-embedding-and-what-can-you-do-with-it-61ba7c05efd8
- https://www.analyticsvidhya.com/blog/2021/06/nlp-sentiment-analysis/
- https://www.freecodecamp.org/news/google-bert-nlp-machine-learning-tutorial/
-https://github.com/dejongyeong/sentiment-analysis
- https://github.com/bentrevett/pytorch-sentiment-analysis
- https://www.freecodecamp.org/news/google-bert-nlp-machine-learning-tutorial/
- https://neptune.ai/blog/category/experiment-tracking
- https://www.datacamp.com/community/tutorials/simplifying-sentiment-analysis-python
- https://www.kaggle.com/iamprateek/sentiment-analysis-using-roberta/notebook
- https://towardsdatascience.com/discover-the-sentiment-of-reddit-subgroup-using-roberta-model-10ab9a8271b8
- https://arxiv.org/pdf/2001.05326.pdf
- https://towardsdatascience.com/bert-for-dummies-step-by-step-tutorial-fb90890ffe03
- https://fasttext.cc/
### models:
-https://huggingface.co/j-hartmann/emotion-english-distilroberta-base
-https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment
- https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment?text=the+was+not+just+pretty+sad+but+not+good+either
-https://www.tensorflow.org/hub
-https://textblob.readthedocs.io/en/dev/classifiers.html#next-steps
-https://annals-csis.org/Volume_24/drp/pdf/20.pdf
-https://www.geeksforgeeks.org/flair-a-framework-for-nlp/
-https://pypi.org/project/vaderSentiment/
-https://github.com/flairNLP/flair
- https://github.com/shahules786/Twitter-Sentiment
### Performance testing:
- https://realpython.com/python-timer/
- https://www.w3schools.com/python/ref_keyword_assert.asp
- https://towardsdatascience.com/multi-class-metrics-made-simple-part-i-precision-and-recall-9250280bddc2
-https://www.lexalytics.com/lexablog/sentiment-accuracy-baseline-testing
- https://therenegadecoder.com/code/how-to-performance-test-python-code/
- https://www.kaggle.com/vipulgandhi/how-to-choose-right-metric-for-evaluating-ml-model
-https://neptune.ai/blog/hyperparameter-tuning-in-python-a-complete-guide-2020
- https://stackoverflow.com/questions/44734297/how-to-profile-python-3-5-code-line-by-line-in-jupyter-notebook-5
- https://www.youtube.com/watch?v=HW29067qVWk
-https://www.youtube.com/watch?v=PfVxFV1ZPnk&t=2958s
- https://stackoverflow.com/questions/419163/what-does-if-name-main-do
-https://medium.com/@bedigunjit/simple-guide-to-text-classification-nlp-using-svm-and-naive-bayes-with-python-421db3a72d34

### Datasets:
- https://analyticsdrift.com/top-15-datasets-for-sentiment-analysis-with-significant-citations/
- https://www.kaggle.com/kazanova/sentiment140
- https://neptune.ai/blog/document-classification-small-datasets
- https://stackoverflow.com/questions/42467316/python3-how-to-read-the-txt-bz2-file
-https://www.kaggle.com/laowingkin/amazon-fine-food-review-sentiment-analysis
-https://www.kaggle.com/suzanaiacob/sentiment-analysis-of-the-yelp-reviews-data?select=yelp_academic_dataset_tip.json
-https://medium.com/hackerdawn/imdb-review-sentiment-analysis-using-logistic-regression-d7878ee01947
- https://medium.com/analytics-vidhya/performing-sentiment-analysis-on-yelp-restaurant-reviews-962334d6336d
-https://www.kaggle.com/parthsharma5795/comprehensive-twitter-airline-sentiment-analysis
- https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/data
